In [1]:
## imports and fixed parameters

import ipywidgets as widgets
from IPython.display import display, IFrame
from dome_connector import Connector
from cmcl_jobsender import JobSender

DOME_URL = 'https://cmcl.dome40.io/'  # DOME platform URL
API_KEY = "123456"  # hard-coded for now
IFRAME_URL = 'http://192.168.1.171:4242/visualisation'
CMCL_URL = 'http://192.168.1.171:4242/'


Textarea(value='', description='Text:', placeholder='Type something')

Button(button_style='success', description='Submit', icon='check', style=ButtonStyle(), tooltip='Submit')

In [ ]:
# IFrame of visualisation

# Create an Output widget to capture the IFrame
vis = widgets.Output(height='800px')

# Define a function to render the IFrame inside the Output widget
@vis.capture()
def render_iframe():
    display(IFrame(src=IFRAME_URL, width='100%', height='800px'))

# Call the function to render the IFrame
render_iframe()

In [ ]:
# search DOME for emission data - backend

# create connectors

emission_connector = Connector(DOME_URL,API_KEY,"CMCL")

def parse_emission_data(json_emission):
    dict_emission = {}
    metadata = json_emission['metadata']
    data = json_emission['data']
    for i in range(len(metadata)):
        list_output = (split_string(
            metadata[i]['Title'], ['emission in', 'at']))
        scope = data[i]['Scope']
        label = f"{list_output[1]}: {scope}"
        timestamp = list_output[2]
        if label not in dict_emission:
            dict_emission[label] = [timestamp]
        else:
            if timestamp not in dict_emission[label]:
                dict_emission[label].append(timestamp)
    return dict_emission


def split_string(raw_string, list_delimiter):
    list_output = []
    for delimiter in list_delimiter:
        output = raw_string.split(delimiter)[0].strip()
        list_output.append(output)
        raw_string = raw_string.replace(output, "").replace(delimiter, "")
    list_output.append(raw_string.strip())
    return list_output

def get_emission(search_string):
    search_string = search_string or "ship"
    return parse_emission_data(emission_connector.get_data(search_string))

In [ ]:
# search DOME for emission data - frontend

# Create a text area widget
text_area_dome_emission = widgets.Text(
    placeholder="""Name of a location""",
    description="""Search for ship emission simulation data:""",
    layout=widgets.Layout(flex_flow='column'),
    style={'description_width': 'initial'},
    disabled=False
)

# Create a button widget
button_dome_emission = widgets.Button(
    description='Submit',
    button_style='success',
    tooltip='Submit',
    icon='check'
)

def print_with_indent(obj, indent=0):
    if isinstance(obj, dict):
        for key, value in obj.items():
            print('  ' * indent + str(key) + ':')
            print_with_indent(value, indent + 1)
    elif isinstance(obj, list):
        for index, item in enumerate(obj):
            print('  ' * indent + f"[{index}]:")
            print_with_indent(item, indent + 1)
    else:
        print('  ' * indent + str(obj))

# Define a function to handle the button click event
def click_button_dome_emission(b):

    with output_dome_emission:
        output_dome_emission.clear_output()
        # Print the text area content
        print_with_indent(get_emission(text_area_dome_emission.value))


output_dome_emission = widgets.Output(layout=widgets.Layout(max_height='100px'))

# Set the event handler for the button click event
button_dome_emission.on_click(click_button_dome_emission)

In [ ]:
# submit job to CMCL server

old_job_sender = JobSender(CMCL_URL)

# Create a text area widget
text_area_old_job_label = widgets.Text(
    placeholder="""Name of location.""",
    description="""Name of location.""",
    layout=widgets.Layout(flex_flow='row'),
    style={'description_width': 'initial'},
    disabled=False
)

text_area_old_job_step = widgets.Text(
    placeholder="""Number of steps.""",
    description="""Number of steps.""",
    layout=widgets.Layout(flex_flow='row'),
    style={'description_width': 'initial'},
    disabled=False
)

# Create a button widget
button_old_job = widgets.Button(
    description='Submit',
    button_style='success',
    tooltip='Submit',
    icon='check'
)

# Define a function to handle the button click event
def click_button_old_job(b):

    with output_old_job:
        output_old_job.clear_output()
        label = text_area_old_job_label.value
        num_step = int(text_area_old_job_step.value)
        
        print_with_indent(old_job_sender.run_simulation(label,None,num_step))

output_old_job = widgets.Output(layout=widgets.Layout(max_height='100px'))

# Set the event handler for the button click event
button_old_job.on_click(click_button_old_job)

In [ ]:
# final-layout

label_dome_emission = widgets.HTML(
value='<p style="font-size:36px; font-weight:bold;text-align:center">Search DOME for emission simulation data</p>'
)

label_old_job = widgets.HTML(
value='<p style="font-size:36px; font-weight:bold;text-align:center">Send update request to existing simulation</p>'
)

# Arrange widgets in each column using VBox
vbox_dome_emission = widgets.VBox(
    [label_dome_emission, text_area_dome_emission, button_dome_emission, output_dome_emission],
    layout=widgets.Layout(width='50%', align_items='center'))
vbox_old_job = widgets.VBox(
    [label_old_job, text_area_old_job_label, text_area_old_job_step,
        button_old_job, output_old_job],
    layout=widgets.Layout(width='50%', align_items='center'))

# Create a horizontal box to hold the widgets
interactive = widgets.HBox(
    [vbox_dome_emission, vbox_old_job], layout=widgets.Layout(width='100%'))



table = widgets.VBox([vis, interactive], layout=widgets.Layout(
    align_items='center'), width='100%')

display(table)